# 1.Feature Engineering

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
## data ingestion
#import libraries and creating connection with Mongodb
import pymongo
import json
client = pymongo.MongoClient("localhost")
print(client)
#defining database and collection
db = client["Regression"]
collection = db["household_power_consumption"]
#Fetching data from database
data = list(collection.find({},{'_id':0}))
df = pd.DataFrame(data)
df

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,19/7/2007,09:28:00,0.210,0.084,228.560,1.000,0.000,0.000,0.0
1,15/12/2007,07:54:00,3.122,0.132,242.140,12.800,0.000,1.000,18.0
2,1/5/2009,05:22:00,1.830,0.458,242.120,7.800,0.000,1.000,18.0
3,2/5/2010,16:02:00,1.754,0.188,239.060,7.400,0.000,0.000,18.0
4,3/7/2010,23:52:00,1.272,0.250,241.000,5.400,0.000,0.000,1.0
...,...,...,...,...,...,...,...,...,...
14995,19/8/2010,06:49:00,?,?,?,?,?,?,NaN
14996,8/1/2009,21:22:00,2.266,0.066,241.810,9.400,0.000,0.000,18.0
14997,3/9/2009,20:47:00,0.524,0.062,238.840,2.200,0.000,0.000,0.0
14998,2/9/2010,12:33:00,1.378,0.150,237.610,5.800,0.000,1.000,18.0


In [3]:
df.shape

(15000, 9)

In [4]:
df.dtypes

Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

### Filling null values

In [5]:
df.isnull().sum()

Date                       0
Time                       0
Global_active_power        0
Global_reactive_power      0
Voltage                    0
Global_intensity           0
Sub_metering_1             0
Sub_metering_2             0
Sub_metering_3           187
dtype: int64

We have null values in Sub_metering_3 feature which is of float64 type we can fill zero if there is no any consumption

In [6]:
df["Sub_metering_3"].fillna(0,inplace=True)

In [7]:
df.isnull().sum()

Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

Now we dont have any null values

## 1.2 Convert Data Types

In [8]:
df.sample()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
8887,23/8/2009,10:09:00,0.232,0.128,240.400,1.000,0.000,0.000,0.0


All columns contains floating numerical values except date

In [9]:
cols = df.columns
cols

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [10]:
#Converting data types of columns
for col in cols[2:]:
    df[col] = df[col].replace("?","0")
    df[col] = df[col].astype("float64")

In [11]:
df.dtypes

Date                      object
Time                      object
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtype: object

In [12]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,19/7/2007,09:28:00,0.210,0.084,228.56,1.0,0.0,0.0,0.0
1,15/12/2007,07:54:00,3.122,0.132,242.14,12.8,0.0,1.0,18.0
2,1/5/2009,05:22:00,1.830,0.458,242.12,7.8,0.0,1.0,18.0
3,2/5/2010,16:02:00,1.754,0.188,239.06,7.4,0.0,0.0,18.0
4,3/7/2010,23:52:00,1.272,0.250,241.00,5.4,0.0,0.0,1.0


In [13]:
##Creating month feature as it can effect the consumtion rate of electricity
df["month"] = pd.to_datetime(df["Date"],infer_datetime_format=True).dt.month

In [14]:
### Creating total metering feature
df["total_metering"] = df["Sub_metering_1"] + df["Sub_metering_2"] + df["Sub_metering_3"]

In [15]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,month,total_metering
0,19/7/2007,09:28:00,0.210,0.084,228.56,1.0,0.0,0.0,0.0,7,0.0
1,15/12/2007,07:54:00,3.122,0.132,242.14,12.8,0.0,1.0,18.0,12,19.0
2,1/5/2009,05:22:00,1.830,0.458,242.12,7.8,0.0,1.0,18.0,5,19.0
3,2/5/2010,16:02:00,1.754,0.188,239.06,7.4,0.0,0.0,18.0,5,18.0
4,3/7/2010,23:52:00,1.272,0.250,241.00,5.4,0.0,0.0,1.0,7,1.0


In [16]:
df.dtypes

Date                      object
Time                      object
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
month                      int64
total_metering           float64
dtype: object

All features are converted into respective types

## Database operations

### Restoring data into database

In [17]:
## Deleting old data
db = client["Regression"]
if "household_power_consumption" in db.list_collection_names():
    db.drop_collection("household_power_consumption")
#defining collection
collection = db["household_power_consumption"]
#Storing data into database
res = collection.insert_many(json.loads(df.to_json(orient="records")))
print(res)